In [97]:
#just need to extract the time for the LED pulse and produce a CSV with this in.
#LED square pulse should be going up to 2.9V but triggered when 960mV - find time on oscilloscope when this is true and the index of this

In [98]:
import sys
import time
import matplotlib.pyplot as plt 
import numpy as np 
import tekwfm
import os
from os.path import exists
from scipy import signal
from scipy import integrate
from scipy.optimize import curve_fit
from scipy.special import gamma
import matplotlib.cm as cm
from collections import Counter

import gc
import re
import tempfile
from collections import defaultdict, namedtuple
from pathlib import Path
import pandas as pd

In [99]:
doVerbose = False

#import the wfm file
filename_base = "030226_spe_2000_500_3_on_ch2"


#saving the processed file

folder_name = "PMT7_SPE_PROC"
file_name = "030226_spe_2000_500_3_on_ch2_processed.csv" 
file_path = os.path.join(os.getcwd(), folder_name, file_name)


startEvent = 0
nEvents = 500


volts, tstart, tscale, tfrac, tdatefrac, tdate = tekwfm.read_wfm(filename_base+".wfm")
if doVerbose:
    print('>>>>', volts, tstart, tscale, tfrac, tdatefrac, tdate)

samples = volts.shape

print(len(volts))
print(tscale)
print('samples', samples[0])
tstop = samples[0]*tscale+tstart
sampleTimes = [tstart+x*tscale for x in range(samples[0])]

#already positive, don't need to flip like with CH1.
tempVolts = [volts[:, event] for event in range(startEvent, startEvent + nEvents)] # all events, array of array of voltages



12500
8e-11
samples 12500


In [100]:
def first_crossing_time(tempVolts, sampleTimes, threshold=0.960):

    for i in range(1, len(tempVolts)):
        if tempVolts[i-1] < threshold <= tempVolts[i]:
            #linear interpolation
            y1, y2 = tempVolts[i-1], tempVolts[i]
            x1, x2 = sampleTimes[i-1], sampleTimes[i]
    
            if (y2 - y1)==0:
                return None, None, None
            frac = (threshold - y1) / (y2 - y1)
            if not (0<=frac<=1):
                return None, None, None
        
            first_cross = x1 + frac * (x2 - x1)

            return first_cross
    
    #if no crossing found
    return None

def cross_index(first_cross, sampleTimes):
    if first_cross is None:
        return len(sampleTimes) - 1
    else:
        time_difference = np.abs(np.array(sampleTimes) - first_cross)
        cross_index = np.argmin(time_difference)
        return cross_index



In [101]:
#for all events
cross_times = []
cross_indices = []

for tempVolt in tempVolts:
    t_cross = first_crossing_time(tempVolt, sampleTimes)
    i_cross = cross_index(t_cross, sampleTimes)
    cross_times.append(t_cross)
    cross_indices.append(i_cross)

print(cross_times)
print(cross_indices)

[None, np.float64(-3.9403870967741956e-08), np.float64(-3.8166285714285717e-07), np.float64(-2.4546117647058827e-07), np.float64(-2.2569818181818182e-07), np.float64(-7.864000000000002e-08), np.float64(-1.442594594594595e-07), np.float64(-2.1382e-07), np.float64(-3.149824e-07), None, np.float64(-1.2468965517241383e-07), None, None, None, None, np.float64(-3.003644444444445e-07), np.float64(-1.8529000000000001e-07), np.float64(-3.9017999999999996e-07), np.float64(-2.372e-07), np.float64(-7.479333333333334e-08), np.float64(-2.3649066666666668e-07), np.float64(-3.23018e-07), None, np.float64(-9.878956521739132e-08), np.float64(-3.311294117647059e-07), np.float64(-3.6489290322580647e-07), np.float64(-1.2281379310344829e-07), None, None, np.float64(-3.474030769230769e-07), None, np.float64(-3.1483764705882355e-07), None, np.float64(-1.6405333333333334e-07), np.float64(-2.3758702702702704e-07), np.float64(-7.653925925925926e-08), None, np.float64(-3.0169391304347826e-07), np.float64(-3.32552

In [102]:
#produce the csv file

df = pd.DataFrame({
            'crossing_trigger': cross_times,
            'crossing_index': cross_indices,
            })

df.to_csv(file_path, sep=',', encoding='utf-8-sig', index=True, header=True)

print('CSV done!')

CSV done!
